# Introduction


In [3]:
# Install the required packages (pathway and bokeh)
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from bokeh.plotting import figure, column, show, output_notebook
output_notebook()     # Enables Bokeh to render plots

# Step 1: Importing and Preprocessing the Data

In [5]:
# Load dataset
df = pd.read_csv('dataset.csv')
# Display column names and first few rows to understand the data
print(df.columns)
print(df.head())

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   

# Feature Scaling

In [6]:
# Calculate occupancy rate as a new feature (percentage of parking slots occupied)
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']

# Map vehicle types to numerical weights (assumed impact on space or pricing)
vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_map)

# Convert traffic condition to binary values: 'low' → 0, 'high' → 1
traffic_map = {'low': 0, 'high': 1}
df['traffic'] = df['TrafficConditionNearby'].map(traffic_map)

# Convert special day status to binary: 'no' → 0, 'yes' → 1
special_day_map = {'no': 0, 'yes': 1}
df['special_day'] = df['IsSpecialDay'].map(special_day_map)

In [7]:
# Handle missing data (fill NaN values)
df['QueueLength'] = df['QueueLength'].fillna(0)
df['occupancy_rate'] = df['occupancy_rate'].fillna(0)
df['traffic'] = df['traffic'].fillna(0)
df['special_day'] = df['special_day'].fillna(0)
df['vehicle_weight'] = df['vehicle_weight'].fillna(0)

# Step 2: MODEL 1: Baseline Linear Pricing

In [8]:
# Initialize price_model1 column with base price 10
df['price_model1'] = 10
alpha = 2

# Ensure price column is float type for calculations
df['price_model1'] = df['price_model1'].astype(float)

# Iteratively update price
for i in range(1, len(df)):
    df.loc[i, 'price_model1'] = df.loc[i-1, 'price_model1'] + alpha * df.loc[i, 'occupancy_rate']

# Clip prices to stay within the realistic bounds (5 to 20)
df['price_model1'] = df['price_model1'].clip(lower=5, upper=20)

# Step 3: MODEL 2: Demand-Based Pricing

In [9]:
# Define weights for demand components
a, b, c, d, e = 2, 1, 1.5, 2, 1.2

# Compute raw demand score using a weighted formula
df['demand_score'] = (a * df['occupancy_rate'] +
                      b * df['QueueLength'] -
                      c * df['traffic'] +
                      d * df['special_day'] +
                      e * df['vehicle_weight'])

# Normalize demand score between 0 and 1 for scaling
df['normalized_demand'] = (df['demand_score'] - df['demand_score'].min()) / (df['demand_score'].max() - df['demand_score'].min())

# Calculate dynamic pricing: base price 10
df['price_model2'] = 10 * (1 + 0.8 * df['normalized_demand'])

# Clip prices to stay between 5 and 20
df['price_model2'] = df['price_model2'].clip(lower=5, upper=20)

# Step 4: MODEL 3: Competitive Pricing Based on Distance

In [10]:
# Define haversine function to compute distance (in km) between two lat/lon pairs
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat, dlon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# Start with price_model2 as initial price for model 3
df['price_model3'] = df['price_model2'].copy()

# Apply competitive adjustment only if timestamp data
if 'timestamp' in df.columns:
    for t in df['timestamp'].unique():  # Iterate over each unique timestamp
        df_time = df[df['timestamp'] == t]
        for i, row in df_time.iterrows():  # Iterate over parking lots in this time slice
            lat, lon = row['latitude'], row['longitude']
            price = row['price_model2']
            occupancy = row['occupancy']
            capacity = row['capacity']

            # Get other competitor lots at same time step
            competitors = df_time[df_time['lot_id'] != row['lot_id']]

            # Filter competitors within 0.5 km distance
            nearby = competitors[competitors.apply(lambda x: haversine(lat, lon, x['latitude'], x['longitude']) < 0.5, axis=1)]

            # Apply pricing adjustments:
            # If fully occupied and any nearby competitor is cheaper, lower price by 1
            if occupancy >= capacity and (nearby['price_model2'] < price).any():
                df.loc[i, 'price_model3'] = price - 1

            # If all nearby competitors are more expensive, raise price by 1
            elif (nearby['price_model2'] > price).all():
                df.loc[i, 'price_model3'] = price + 1

# Clip prices to stay between 5 and 20
df['price_model3'] = df['price_model3'].clip(lower=5, upper=20)


# Step 3: Visualizing Price Fluctuations with a Bokeh Plot

In [13]:
# Plotting all three models using Bokeh

# Define x-axis values as time steps or row indices
x_values = list(range(len(df)))

# Create Bokeh figure for Model 1: Baseline Linear Pricing
p1 = figure(title='Model 1: Baseline', x_axis_label='Time Step', y_axis_label='Price')
p1.line(x_values, df['price_model1'], line_width=2, color='blue')

# Create Bokeh figure for Model 2: Demand-Based Dynamic Pricing
p2 = figure(title='Model 2: Dynamic', x_axis_label='Time Step', y_axis_label='Price')
p2.line(x_values, df['price_model2'], line_width=2, color='red')

# Create Bokeh figure for Model 3: Competitive Pricing with distance-based adjustments
p3 = figure(title='Model 3: Competitive', x_axis_label='Time Step', y_axis_label='Price')
p3.line(x_values, df['price_model3'], line_width=2, color='yellow')

# Show all three plots stacked vertically
show(column(p1, p2, p3))